# Task 2

### Data Preprocessing
* Resize input images to 64×64×3 or 128×128×3, depending on available computational resources
* Normalize both training and testing image sets

In [2]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from sklearn.model_selection import train_test_split

IMAGE_SIZE = 128
NUM_CHANNELS = 3
NUM_CLASSES = 20
BATCH_SIZE = 32
TEST_SPLIT = 0.2
VALIDATION_SPLIT = 0.1
SEED = 42

DATA_DIR = "coil-20-proc/coil-20-proc"


def parse_class_label_from_filename(filename):
    object_prefix = filename.split("__")[0]
    one_indexed_label = int(object_prefix.replace("obj", ""))
    return one_indexed_label - 1


def load_coil20_images_and_labels(data_dir):
    all_images = []
    all_labels = []
    for filename in sorted(os.listdir(data_dir)):
        if not filename.endswith(".png"):
            continue
        label = parse_class_label_from_filename(filename)
        grayscale_image = Image.open(os.path.join(data_dir, filename))
        rgb_image = grayscale_image.convert("RGB").resize((IMAGE_SIZE, IMAGE_SIZE))
        all_images.append(np.array(rgb_image))
        all_labels.append(label)
    return np.array(all_images), np.array(all_labels)


def normalize_pixel_values(images):
    return images.astype(np.float32) / 255.0


def numpy_to_channels_first_tensor(images):
    return torch.tensor(images).permute(0, 3, 1, 2)


class COIL20Dataset(Dataset):
    def __init__(self, image_tensors, label_tensors):
        self.image_tensors = image_tensors
        self.label_tensors = label_tensors

    def __len__(self):
        return len(self.label_tensors)

    def __getitem__(self, idx):
        return self.image_tensors[idx], self.label_tensors[idx]


raw_images, labels = load_coil20_images_and_labels(DATA_DIR)
print(f"Loaded {raw_images.shape[0]} images of shape {raw_images.shape[1:]}")
print(f"Classes: {np.unique(labels)}")
print(f"Pixel range before normalization: [{raw_images.min()}, {raw_images.max()}]")

normalized_images = normalize_pixel_values(raw_images)
print(f"Pixel range after normalization: [{normalized_images.min():.2f}, {normalized_images.max():.2f}]")

train_images, test_images, train_labels, test_labels = train_test_split(
    normalized_images, labels, test_size=TEST_SPLIT, random_state=SEED, stratify=labels
)
print(f"\nTraining set: {train_images.shape[0]} samples")
print(f"Testing set:  {test_images.shape[0]} samples")

train_image_tensors = numpy_to_channels_first_tensor(train_images)
test_image_tensors = numpy_to_channels_first_tensor(test_images)
train_label_tensors = torch.tensor(train_labels, dtype=torch.long)
test_label_tensors = torch.tensor(test_labels, dtype=torch.long)

train_dataset = COIL20Dataset(train_image_tensors, train_label_tensors)
test_dataset = COIL20Dataset(test_image_tensors, test_label_tensors)

train_subset_size = int((1 - VALIDATION_SPLIT) * len(train_dataset))
val_subset_size = len(train_dataset) - train_subset_size
train_subset, val_subset = random_split(
    train_dataset, [train_subset_size, val_subset_size],
    generator=torch.Generator().manual_seed(SEED)
)

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"\nTrain loader: {len(train_subset)} samples, {len(train_loader)} batches")
print(f"Val loader:   {len(val_subset)} samples, {len(val_loader)} batches")
print(f"Test loader:  {len(test_dataset)} samples, {len(test_loader)} batches")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")

Loaded 1440 images of shape (128, 128, 3)
Classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Pixel range before normalization: [0, 255]
Pixel range after normalization: [0.00, 1.00]

Training set: 1152 samples
Testing set:  288 samples

Train loader: 1036 samples, 33 batches
Val loader:   116 samples, 4 batches
Test loader:  288 samples, 9 batches

Using device: cpu


### Activation Function Design and Comparison
* Use an AI-assistive system to design a novel activation function (reference: ReLU, Swish, GELU, TeLU, etc.)
* Integrate the proposed activation function into the DCNN
* Compare its performance against standard activation functions such as ReLU and GELU

### Model Optimization
Train the model using the following optimization methods:
* Stochastic Gradient Descent (SGD) with momentum
* Adam optimizer

### Training Analysis
Plot training and validation loss curves to demonstrate learning behavior and convergence during training

### Model Evaluation
Evaluate the trained model on the test dataset using the following performance metrics:
* Accuracy
* Precision
* Recall
* F1-score
* Confusion matrix
* ROC curve
* Precision–Recall curve

Present overall performance without thresholding, then also report metrics for predictions with confidence > 0.9.